In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import lsst.daf.butler as dB
import lsst.cp.verify.notebooks.utils as utils
import lsst.afw.display as afwDisplay

In [ ]:
# This cell contains parameters that can be automatically set via the papermill package.
# Examples:
#   papermill --prepare-only -p calibType newBias -p cameraName LSSTCam <input> <output>
#   papermill -p interactive False <input> <output>
interactive = True

# Which repository to use.
repository = '/repo/main/'

# Which calibration type to analyse.
calibType = 'dark'

# Which camera the calibration is for.
cameraName = 'LATISS'

# Which display to use.
displayBackend = 'astrowidgets'

# Which collection the calibration was constructed in.
genCollection = 'u/czw/DM-28920/darkGen.20210707d'

# Which collection containing the verification outputs.
verifyCollection = 'u/czw/DM-28920/verifyDark.20210707d'

In [ ]:
# Get butler and camera
butler = dB.Butler(repository, collections=[verifyCollection, genCollection])
camera = butler.get('camera', instrument=cameraName)

In [ ]:
# Get Run Statistics
runStats = butler.get('verifyDarkStats', instrument=cameraName)
runSuccess = runStats.pop('SUCCESS')

In [ ]:
# Display summary table of tests and failure counts.
utils.failureTable(runStats)

In [ ]:
# Plot focal plane, and display number of amp-level failures per detector.
utils.plotFailures(runStats, camera, scaleFactor=8)

In [ ]:
# This cell may be easier to follow in a new view via the
#     "Create New View for Output" right-click menu.  
afwDisplay.setDefaultBackend(displayBackend)
display = afwDisplay.Display(dims=(1000, 1000))
display.embed()

In [ ]:
# View calibration images:
viewCalibs = interactive
if viewCalibs:
    continueDisplay = True
    for detector in camera:
        detectorId = detector.getId()
        calib = butler.get(calibType, instrument=cameraName, detector=detectorId)
        calibArray = calib.getImage().getArray()

        # Get simple stats
        q25, q50, q75 = np.percentile(calibArray.flatten(), [25, 50, 75])
        sigma = 0.74 * (q75 - q25)
        print(f"Detector: {detector.getName()} Median: {q50}   Stdev: {sigma}")
    
        display.mtv(calib)
        display._scale('linear', (q50 - 3.0 * sigma), (q50 + 3.0* sigma), "")

        continueDisplay, skipNumber = utils.interactiveBlock(f"{calibType} {detector.getName()}", 
                                                             {})
        if continueDisplay is False:
            break

In [ ]:
# This block allows the residual images to be scanned for concerns.
blinkResiduals = interactive
if blinkResiduals:
    continueDisplay = True
    skipNumber = 0
    for exposureId, stats in runStats.items():
        for detector in camera:
            if skipNumber > 0:
                skipNumber -= 1
                continue
        
            detId = detector.getId()
            residual = butler.get('verifyDarkProc', instrument=cameraName, exposure=exposureId, detector=0)
            detStats = butler.get('verifyDarkDetStats', instrument=cameraName, exposure=exposureId, detector=0)
            display.mtv(residual)
            display.scale('linear', 'zscale', None)

            continueDisplay, skipNumber = utils.interactiveBlock(f"{exposureId} {detector.getName()}", detStats)
            if continueDisplay is False:
                break
        if continueDisplay is False:
            break

In [ ]:
# Additional cells follow here.

In [ ]:
# Get data for mean(darkTime) plot.
ampMeans = {}
for detector in camera:
    ampMeans[detector.getName()] = {}
    for amp in detector.getAmplifiers():
        ampMeans[detector.getName()][amp.getName()] = {'DARKTIME': [], 'MEAN': []}

for exposureId, stats in runStats.items():
    dimensionRecord = butler.registry.queryDimensionRecords('exposure', 
                                                            instrument=cameraName, 
                                                            exposure=exposureId)
    darkTime = list(dimensionRecord)[0].dark_time

    for detector in camera:
        detId = detector.getId()
        detStats = butler.get('verifyDarkDetStats', instrument=cameraName, 
                              exposure=exposureId, detector=detId)
  
        for amp in detector.getAmplifiers():
            mean = detStats['AMP'][amp.getName()]['MEAN']
            ampMeans[detector.getName()][amp.getName()]['DARKTIME'].append(darkTime)
            ampMeans[detector.getName()][amp.getName()]['MEAN'].append(mean)

In [ ]:
# Plot mean as a function of exposure time, to confirm the residual is flat.
for detector in camera:
    detName = detector.getName()
    horizontalSpace = 0.5
    verticalSpace = 0.0
    plt.figure(figsize=(8, 8))
    for spacer, amp in enumerate(detector.getAmplifiers()):
        plt.scatter(np.array(ampMeans[detName][amp.getName()]['DARKTIME']) + horizontalSpace * spacer,
                    np.array(ampMeans[detName][amp.getName()]['MEAN']) + verticalSpace * spacer,
                    label=amp.getName())
    plt.xlabel("DarkTime (s)")
    plt.ylabel("Residual Mean (ADU)")
    plt.title(f"{calibType} {cameraName} {detName} {verifyCollection}")
    plt.legend()
    plt.show()